# Topic Detect Model in Zhihu
``(Sentence-BERT + UMAP + HDBSCAN)``

In [19]:
import json
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

# 设置显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

In [11]:
import torch
import umap
import hdbscan
import numpy as np
import re
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from dotenv import load_dotenv
import os

load_dotenv()


class SentenceEmbeddingModel:
    def __init__(self, model_path, device=None):
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = SentenceTransformer(model_path, device=self.device)
    
    def forward(self, sentences, batch_size=64, show_progress=True):
        return self.model.encode(
            sentences,
            batch_size=batch_size,
            show_progress_bar=show_progress,
            normalize_embeddings=True,
            device=self.device
        )


class UMAPReducer:
    def __init__(self, n_components=50, n_neighbors=15, min_dist=0.1, random_state=42):
        self.n_components = n_components
        self.n_neighbors = n_neighbors
        self.min_dist = min_dist
        self.random_state = random_state
        self.reducer = None
        
    def fit_transform(self, embeddings):
        self.reducer = umap.UMAP(
            n_components=self.n_components,
            n_neighbors=self.n_neighbors,
            min_dist=self.min_dist,
            metric='cosine',
            random_state=self.random_state
        )
        return self.reducer.fit_transform(embeddings)
    
    def transform(self, embeddings):
        return self.reducer.transform(embeddings)


class HDBSCANCluster:
    def __init__(self, min_cluster_size=30, min_samples=None):
        self.min_cluster_size = min_cluster_size
        self.min_samples = min_samples or min_cluster_size // 2
        self.clusterer = hdbscan.HDBSCAN(
            min_cluster_size=self.min_cluster_size,
            min_samples=self.min_samples,
            metric='euclidean',
            prediction_data=True,
            cluster_selection_method='eom'
        )
        self.labels_ = None
        self.topic_centers_ = {}
        self.topic_sizes_ = {}
    
    def fit(self, embeddings):
        embeddings = normalize(embeddings)
        self.clusterer.fit(embeddings)
        self.labels_ = self.clusterer.labels_
        self._compute_topic_centers(embeddings)
        return self.labels_
    
    def _compute_topic_centers(self, embeddings):
        topic_vectors = defaultdict(list)
        for vec, label in zip(embeddings, self.labels_):
            if label != -1:
                topic_vectors[label].append(vec)
        
        for label, vecs in topic_vectors.items():
            vecs = np.vstack(vecs)
            center = vecs.mean(axis=0)
            self.topic_centers_[label] = center / np.linalg.norm(center)
            self.topic_sizes_[label] = len(vecs)
    
    def get_topics(self):
        topics = [
            {"topic_id": l, "size": self.topic_sizes_[l], "center": self.topic_centers_[l]}
            for l in self.topic_centers_
        ]
        return sorted(topics, key=lambda x: x['size'], reverse=True)
    
    def predict(self, new_embeddings):
        new_embeddings = normalize(new_embeddings)
        labels, _ = hdbscan.approximate_predict(self.clusterer, new_embeddings)
        return labels


class TopicDetector:
    def __init__(self, model_path=None, device=None, use_umap=True, 
                 umap_components=50, umap_neighbors=25, min_cluster_size=30, min_samples=None):
        self.model_path = model_path or os.getenv('SEN_EMB_MODEL_PATH', '../src/models/text2vec-base-chinese')
        self.embedding_model = SentenceEmbeddingModel(self.model_path, device)
        
        if use_umap:
            self.umap_reducer = UMAPReducer(n_components=umap_components, n_neighbors=umap_neighbors)
        else:
            self.umap_reducer = None
            
        self.cluster_model = HDBSCANCluster(min_cluster_size, min_samples)
        self.embeddings = None
        self.embeddings_reduced = None
        self.labels = None
    
    def fit(self, texts, batch_size=128, show_progress=True):
        print("Step 1: 生成句子嵌入...")
        self.embeddings = self.embedding_model.forward(texts, batch_size, show_progress)
        print(f"  维度: {self.embeddings.shape}")
        
        if self.umap_reducer:
            print("Step 2: UMAP 降维...")
            self.embeddings_reduced = self.umap_reducer.fit_transform(self.embeddings)
        else:
            self.embeddings_reduced = self.embeddings
        
        print("Step 3: HDBSCAN 聚类...")
        self.labels = self.cluster_model.fit(self.embeddings_reduced)
        
        n_clusters = len(set(self.labels)) - (1 if -1 in self.labels else 0)
        n_noise = list(self.labels).count(-1)
        print(f"  主题: {n_clusters}, 噪声: {n_noise} ({n_noise/len(self.labels)*100:.1f}%)")
        
        return self
    
    def fit_predict(self, texts, **kwargs):
        self.fit(texts, **kwargs)
        return self.labels, self.cluster_model.get_topics()
    
    def get_topic_summary(self):
        if self.labels is None:
            return {}
        n_clusters = len(set(self.labels)) - (1 if -1 in self.labels else 0)
        n_noise = list(self.labels).count(-1)
        return {
            "total_samples": len(self.labels),
            "n_topics": n_clusters,
            "n_noise": n_noise,
            "noise_ratio": f"{n_noise / len(self.labels) * 100:.1f}%",
        }
    
    def get_representative_texts(self, texts, topic_id, top_k=3):
        center = self.cluster_model.topic_centers_[topic_id]
        topic_indices = [i for i, l in enumerate(self.labels) if l == topic_id]
        
        embeddings_norm = normalize(self.embeddings_reduced)
        similarities = embeddings_norm[topic_indices] @ center
        top_idx = np.argsort(similarities)[-top_k:][::-1]
        
        return [(topic_indices[i], texts[topic_indices[i]], similarities[i]) for i in top_idx]
    
    def extract_keywords_tfidf(self, texts, topic_id, top_k=5):
        topic_indices = [i for i, l in enumerate(self.labels) if l == topic_id]
        topic_texts = [texts[i] for i in topic_indices]
        
        if len(topic_texts) < 2:
            return []
        
        vectorizer = TfidfVectorizer(max_features=1000, token_pattern=r'(?u)\b\w+\b')
        try:
            tfidf_matrix = vectorizer.fit_transform(topic_texts)
            feature_names = vectorizer.get_feature_names_out()
            mean_scores = tfidf_matrix.mean(axis=0).A1
            top_indices = mean_scores.argsort()[-top_k:][::-1]
            return [feature_names[i] for i in top_indices]
        except:
            return []


def clean_text(text, max_len=150):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'#[^#\s]+#?', '', text)
    return text[:max_len] + '...' if len(text) > max_len else text.strip()

In [12]:
data_dir = Path("../data")
data_file = data_dir / "zhihu_ring_data_20260225_senti.json"
circles_file = data_dir / "zhihu_ai_circles.json"

with open(data_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

with open(circles_file, 'r', encoding='utf-8') as f:
    circles_data = json.load(f)

ring_to_name = {c['ring_id']: c['name'] for c in circles_data}
df = pd.DataFrame(data)
df['ring_name'] = df['ring_id'].map(ring_to_name).fillna(df['ring_id'])

print(f"总数据量: {len(data)} 条")
print(f"圈子分布:\n{df['ring_name'].value_counts().head(10)}")

总数据量: 3009 条
圈子分布:
ring_name
AI与人类未来          466
AI 创投生态圈         263
科研 AI Hub        255
算法研究所            253
AI写作研究所          246
AI Coding 探索舰    241
AI 工具测评中心        237
AI 时代的我们         226
AI 安全研究所         214
DeepSeek 深潜舱     168
Name: count, dtype: int64


In [13]:
import os

# 获取项目根目录（notebook 目录的上级目录）
PROJECT_ROOT = Path(os.path.abspath('.')).parent
model_path = PROJECT_ROOT / 'src' / 'models' / 'text2vec-base-chinese'

data_dir = Path("../data")
data_file = data_dir / "zhihu_ring_data_20260225_senti.json"
circles_file = data_dir / "zhihu_ai_circles.json"

with open(data_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

with open(circles_file, 'r', encoding='utf-8') as f:
    circles_data = json.load(f)

ring_to_name = {c['ring_id']: c['name'] for c in circles_data}
df = pd.DataFrame(data)
df['ring_name'] = df['ring_id'].map(ring_to_name).fillna(df['ring_id'])

print(f"总数据量: {len(data)} 条")
print(f"模型路径: {model_path}")
print(f"圈子分布:\n{df['ring_name'].value_counts().head(10)}")

总数据量: 3009 条
模型路径: d:\Projects\nlp-public-opinion-analysis\src\models\text2vec-base-chinese
圈子分布:
ring_name
AI与人类未来          466
AI 创投生态圈         263
科研 AI Hub        255
算法研究所            253
AI写作研究所          246
AI Coding 探索舰    241
AI 工具测评中心        237
AI 时代的我们         226
AI 安全研究所         214
DeepSeek 深潜舱     168
Name: count, dtype: int64


In [14]:
texts = df['content'].tolist()
print(f"总文本数: {len(texts)} 条, 平均长度: {np.mean([len(t) for t in texts]):.0f} 字符")

# 使用优化后的 TopicDetector
detector = TopicDetector(model_path=str(model_path), min_cluster_size=30, umap_neighbors=25)
labels, topics = detector.fit_predict(texts)

summary = detector.get_topic_summary()
print(f"\n聚类结果: {summary}")

for t in topics[:10]:
    print(f"Topic {t['topic_id']}: {t['size']} 条")

总文本数: 3009 条, 平均长度: 299 字符
Step 1: 生成句子嵌入...


Batches: 100%|██████████| 24/24 [02:20<00:00,  5.84s/it]


  维度: (3009, 768)
Step 2: UMAP 降维...
Step 3: HDBSCAN 聚类...
  主题: 9, 噪声: 1613 (53.6%)

聚类结果: {'total_samples': 3009, 'n_topics': 9, 'n_noise': 1613, 'noise_ratio': '53.6%'}
Topic 1: 400 条
Topic 7: 399 条
Topic 0: 229 条
Topic 3: 144 条
Topic 5: 67 条
Topic 2: 44 条
Topic 4: 44 条
Topic 6: 37 条
Topic 8: 32 条


In [15]:
for t in topics[:10]:
    print(f"\n--- Topic {t['topic_id']} ({t['size']} 条) ---")
    
    reps = detector.get_representative_texts(texts, t['topic_id'])
    for idx, text, sim in reps:
        print(f"  [{clean_text(text)}] ({sim:.2f})")
    
    keywords = detector.extract_keywords_tfidf(texts, t['topic_id'], top_k=5)
    print(f"  关键词: {', '.join(keywords)}")


--- Topic 1 (400 条) ---
  [为什么我还是无法理解Transformer？Transformer到底是什么？ Transformer是一种神经网络结构，以自注意力机制为核心，摒弃传统循环与卷积结构，实现计算并行化，高效捕捉序列中的长距离依赖关系。这一创新结构有效解决了传统模型的效率难题，大幅提升前向传播与特征建模的效率。其实用性...] (1.00)
  [后ControlNet时代：ControlNet之后的可控生成研究还有哪些新范式？ 🙋‍♂️ 专题分享更新啦～本期主题主要围绕后ControlNet时代。从Diffusion Guidance到DiT的可控生成的一些技术演进分享～欢迎交流～ 🚩 ControlNet定义的「Add-on」范式在SD ...] (1.00)
  [40多页大模型架构综述，Transformer模型如何进行优化 Transformer架构存在计算资源消耗高、训练与部署难度大的问题。 这篇综述系统性地探讨了突破Transformer局限的高效架构，以及详细的介绍了现代的LLM的架构： 线性与稀疏序列建模（如线性注意力、状态空间模型） 高效全注意力...] (1.00)
  关键词: 阅读全文, 链接, 2025, 1, 2

--- Topic 7 (399 条) ---
  [现在我发现AI比大部分剪人更像人，更会说人话 当代艺术、精英主义与慕强：为什么在简中语境里，现代总是被跳过这圆桌怎么全是深病，没用过但也不喜欢剪中类人味 如何通过 AI 建立个人知识库，让科研学习更高效？] (1.00)
  [现在我发现AI比大部分剪人更像人，更会说人话 当代艺术、精英主义与慕强：为什么在简中语境里，现代总是被跳过这圆桌怎么全是深病，没用过但也不喜欢剪中类人味 如何通过 AI 建立个人知识库，让科研学习更高效？… 阅读全文​] (1.00)
  [懂你，AI有多细心？ 一场与AI的对话让饺子馆崩溃的我完成自愈，我把这个对话记录下来了，14万字，叫做《自愈之路——源自一场与AI的对话》。一座未开发的矿山，需要挖掘，需要提纯。在我整理的过程中，它一直陪伴在身边，帮我挖掘“问题背后的问题”，给我灵感，给我提醒，给我鼓励，给我校稿，给我示例。 但是，...] (1.00)
  关键词: 阅读全文, ai, 链接, 1, 2



In [16]:
## 参数对比实验

results = []

for min_cluster_size in [20, 30, 50]:
    for umap_neighbors in [15, 25]:
        det = TopicDetector(
            model_path=str(model_path),
            min_cluster_size=min_cluster_size,
            umap_neighbors=umap_neighbors
        )
        lbls, tops = det.fit_predict(texts, show_progress=False)
        n_clusters = len(set(lbls)) - (1 if -1 in lbls else 0)
        n_noise = list(lbls).count(-1)
        results.append({
            'min_cluster_size': min_cluster_size,
            'umap_neighbors': umap_neighbors,
            'n_topics': n_clusters,
            'noise': n_noise,
            'noise_ratio': f"{n_noise/len(lbls)*100:.1f}%"
        })

print("\n参数对比结果:")
print(f"{'min_cluster_size':<15} {'umap_neighbors':<15} {'n_topics':<10} {'noise':<10} {'noise_ratio':<12}")
print("-" * 65)
for r in results:
    print(f"{r['min_cluster_size']:<15} {r['umap_neighbors']:<15} {r['n_topics']:<10} {r['noise']:<10} {r['noise_ratio']:<12}")

Step 1: 生成句子嵌入...
  维度: (3009, 768)
Step 2: UMAP 降维...
Step 3: HDBSCAN 聚类...
  主题: 2, 噪声: 186 (6.2%)
Step 1: 生成句子嵌入...
  维度: (3009, 768)
Step 2: UMAP 降维...
Step 3: HDBSCAN 聚类...
  主题: 10, 噪声: 1438 (47.8%)
Step 1: 生成句子嵌入...
  维度: (3009, 768)
Step 2: UMAP 降维...
Step 3: HDBSCAN 聚类...
  主题: 10, 噪声: 1666 (55.4%)
Step 1: 生成句子嵌入...
  维度: (3009, 768)
Step 2: UMAP 降维...
Step 3: HDBSCAN 聚类...
  主题: 9, 噪声: 1613 (53.6%)
Step 1: 生成句子嵌入...
  维度: (3009, 768)
Step 2: UMAP 降维...
Step 3: HDBSCAN 聚类...
  主题: 3, 噪声: 1265 (42.0%)
Step 1: 生成句子嵌入...
  维度: (3009, 768)
Step 2: UMAP 降维...
Step 3: HDBSCAN 聚类...
  主题: 3, 噪声: 1268 (42.1%)

参数对比结果:
min_cluster_size umap_neighbors  n_topics   noise      noise_ratio 
-----------------------------------------------------------------
20              15              2          186        6.2%        
20              25              10         1438       47.8%       
30              15              10         1666       55.4%       
30              25              9     

In [18]:
## 主题与圈子的交叉分析

df_result = df.copy()
df_result['topic_id'] = labels

# 主题与圈子的交叉表
cross_tab = pd.crosstab(df_result['topic_id'], df_result['ring_name'])
print("\n主题在各圈子的分布:")
print(cross_tab)

# 每个主题最主要的圈子
print("\n每个主题的主要圈子:")
for topic in sorted(df_result['topic_id'].unique()):
    if topic == -1:
        continue
    topic_data = df_result[df_result['topic_id'] == topic]
    top_rings = topic_data['ring_name'].value_counts().head(3)
    print(f"\nTopic {topic} ({len(topic_data)} 条):")
    for ring, count in top_rings.items():
        print(f"  {ring}: {count} 条 ({count/len(topic_data)*100:.1f}%)")


主题在各圈子的分布:
ring_name  AI Coding 探索舰  AI Talent Hub  AI infra  AI 创投生态圈  AI 安全研究所  \
topic_id                                                                
-1                   128             24        86       182       122   
 0                    77              3         5         8         3   
 1                     9              1        25         2         7   
 2                     2              0         1         7         4   
 3                     3              3        15        33        10   
 4                     1              0         0         5        31   
 5                     0              0         0         1         0   
 6                     0              0         0         1         0   
 7                    21              3         7        23        37   
 8                     0              0         1         1         0   

ring_name  AI 工具测评中心  AI 时代的我们  AI 顶会联盟  AI与人类未来  AI写作研究所  DeepSeek 深潜舱  \
topic_id                            